In [3]:
! pip install -r requirements.txt --quiet

In [ ]:
from semantic_kernel import Kernel
from user_plugins import WeatherPlugin
from semantic_kernel.agents import ChatCompletionAgent
from dotenv import load_dotenv
from os import environ
from azure.core.credentials import AzureKeyCredential

load_dotenv(override=True)

kernel = Kernel()



## 🔍 Tracing with Azure AI Foundry

To enable tracing, use the `AzureAIInferenceChatCompletion` connector from Semantic Kernel. This connector automatically logs telemetry data that can be visualized in the Azure AI Foundry Tracing UI.

🔗 [View traces in Azure AI Foundry](https://learn.microsoft.com/en-us/semantic-kernel/concepts/enterprise-readiness/observability/telemetry-with-azure-ai-foundry-tracing#use-the-azure-ai-inference-connector)

> ⚠️ If traces do not appear in the UI, you can implement a custom wrapper around the `AzureChatCompletion` connector to enable telemetry manually.


In [12]:
## Use the Azure AI Inference connector

from semantic_kernel.connectors.ai.azure_ai_inference import AzureAIInferenceChatCompletion
from azure.ai.inference.aio import ChatCompletionsClient

deployment_name=environ["AZURE_OPENAI_MODEL"]
endpoint=environ["AZURE_OPENAI_ENDPOINT"]




In [13]:
kernel.add_service(AzureAIInferenceChatCompletion(
    ai_model_id=deployment_name,
    client=ChatCompletionsClient(
        endpoint=f"{str(endpoint).strip('/')}/openai/deployments/{deployment_name}",
        credential=AzureKeyCredential(environ["AZURE_OPENAI_API_KEY"]),
        credential_scopes=["https://cognitiveservices.azure.com/.default"],
    ),
))

> ⏳ It may take a few minutes for the traces to show up on the UI.

In [14]:
kernel.add_plugin(WeatherPlugin, plugin_name="Weather")

KernelPlugin(name='Weather', description=None, functions={'get_sunrise_sunset': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='get_sunrise_sunset', plugin_name='Weather', description='Call to get the sunrise and sunset for a given location.', parameters=[KernelParameterMetadata(name='location', description='The location to get the sunrise and sunset', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'The location to get the sunrise and sunset'}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='Any', is_required=True, type_object=None, schema_data={'type': 'object'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x00000279A2432120>, streaming_duration_histogram

In [15]:
# Create the agent
agent = ChatCompletionAgent(
    kernel=kernel, 
    name="WeatherAgent", 
    instructions=(
        "You are an intelligent weather assistant. "
        "Use the available plugins and tools to answer questions about the weather, "
        "including forecasts, temperatures, and conditions. "
        "Always include temperatures in both Celsius and Fahrenheit. "
        "Be concise, friendly, and helpful. If you're unsure about a location, ask for clarification."
    )
)

In [16]:
response = await agent.get_response(messages="What's the weather like in sf today?")
print(response)

San Francisco is sunny today with a temperature of 90°F (32°C). Enjoy the warm weather!


## [Handling Intermediate Messages with a ChatCompletionAgent](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/chat-completion-agent?pivots=programming-language-python#handling-intermediate-messages-with-a-chatcompletionagent)

The Semantic Kernel ChatCompletionAgent is designed to invoke an agent that fulfills user queries or questions. During invocation, the agent may execute tools to derive the final answer. To access intermediate messages produced during this process, callers can supply a callback function that handles instances of FunctionCallContent or FunctionResultContent.



In [20]:

from semantic_kernel.contents import AuthorRole, ChatMessageContent, FunctionCallContent, FunctionResultContent


# Define a list to hold callback message content
intermediate_steps: list[ChatMessageContent] = []

# Define an async method to handle the `on_intermediate_message` callback
async def handle_intermediate_steps(message: ChatMessageContent) -> None:
    intermediate_steps.append(message)



user_inputs = [
    "Hello", 
    "What's the weather like in sf today?", 
    "What time is the sunrise?",
    "Thank you",
]

thread = None

# Generate the agent response(s)
for user_input in user_inputs:
    print(f"# {AuthorRole.USER}: '{user_input}'")
    async for response in agent.invoke(
        messages=user_input,
        thread=thread,
        on_intermediate_message=handle_intermediate_steps,
    ):
        thread = response.thread
        print(f"# {response.name}: {response.content}")

# Delete the thread when it is no longer needed
await thread.delete() if thread else None

# Print the intermediate steps
print("\nIntermediate Steps:")
for msg in intermediate_steps:
    if any(isinstance(item, FunctionResultContent) for item in msg.items):
        for fr in msg.items:
            if isinstance(fr, FunctionResultContent):
                print(f"Function Result:> {fr.result} for function: {fr.name}")
    elif any(isinstance(item, FunctionCallContent) for item in msg.items):
        for fcc in msg.items:
            if isinstance(fcc, FunctionCallContent):
                print(f"Function Call:> {fcc.name} with arguments: {fcc.arguments}")
    else:
        print(f"{msg.role}: {msg.content}")

# AuthorRole.USER: 'Hello'
# WeatherAgent: Hi there! How can I assist you with the weather today? 😊
# AuthorRole.USER: 'What's the weather like in sf today?'
# WeatherAgent: The current weather in San Francisco is foggy, with a temperature of 60°F (15°C). Let me know if you need more details! 🌁
# AuthorRole.USER: 'What time is the sunrise?'
# WeatherAgent: The sunrise in San Francisco today was at 6:05 AM, and the sunset will be at 8:15 PM. Let me know if you'd like more updates! 🌅
# AuthorRole.USER: 'Thank you'
# WeatherAgent: You're very welcome! Have a fantastic day! 😊

Intermediate Steps:
Function Call:> Weather-get_weather with arguments: {"location":"San Francisco"}
Function Result:> It's 60 degrees and foggy. for function: Weather-get_weather
Function Call:> Weather-get_sunrise_sunset with arguments: {"location":"San Francisco"}
Function Result:> Sunrise: 6:05 A.M, Sunset: 8:15 P.M for function: Weather-get_sunrise_sunset
